# Severity Model - Calculo de Umbrales por Zona

Este notebook calcula los umbrales de mean_delay para cada zona logistica.

**Output**: Para cada zona, valores de mean_delay donde activar:
- Low Delay
- Preventive
- Containment I
- Containment II
- Inoperable

In [2]:
# Imports
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.patches import Patch

# Configuracion de visualizacion
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', 50)

# Imports del proyecto
from config.settings import STRESS_WEIGHTS, STAGE_STRESS_PERCENTILES, TABLES, get_start_date, COUNTRY_CODE
from src.data_extraction import extract_all_data
from src.feature_engineering import build_stress_curves
from src.stage_calculator import calculate_zone_thresholds, export_thresholds_for_config

## 1. Extraccion de Datos

In [3]:
# Extraer todos los datos de BigQuery
data = extract_all_data()

/Users/guidod/Desktop/Severity/venv/lib/python3.13/site-packages/google/auth/_default.py:114: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


EXTRACCIÓN DE DATOS - SEVERITY MODEL
País: AR
Período: últimos 90 días
Extrayendo datos de stress por nivel de delay desde 2025-11-27...
  -> Excluyendo fechas: 2025-12-24, 2025-12-25, 2025-12-31, 2026-01-01
  -> Join por MINUTO (timezone UTC -> Argentina)


/Users/guidod/Desktop/Severity/venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


  -> 2,772 registros (zona, delay_minute) extraídos
Extrayendo información de zonas...
  -> 127 zonas encontradas
Extrayendo configuración actual de stages...
  -> 43 ciudades con configuración
EXTRACCIÓN COMPLETADA


## 2. Feature Engineering - Curvas de Stress con Suavizado

In [4]:
# Construir curvas de stress (incluye suavizado Savitzky-Golay)
df_stress = build_stress_curves(data)

print(f"\nColumnas disponibles: {df_stress.columns.tolist()}")
print(f"\nZonas totales: {df_stress['zone_id'].nunique()}")
print(f"Registros totales: {len(df_stress)}")


FEATURE ENGINEERING - STRESS CURVES
Aplicando filtros iniciales de delay...
  -> Filtro max delay <= 30: 0 puntos removidos
  -> Recorte por percentil ponderado P99: 658 puntos removidos
  -> Total puntos removidos por filtros de delay: 658
Calculando baselines por zona (ponderado por ordenes)...
  -> Filtrando puntos con menos de 30 ordenes
  -> Baselines calculados para 123 zonas
Calculando stress scores...
  -> Normalizando stress POR ZONA (solo puntos con >= 30 ordenes)
  -> Stress scores calculados para 2,114 registros
  -> 49 puntos con < 30 ordenes (marcados pero incluidos)
Interpolando valores faltantes...
  -> 79 puntos interpolados
Aplicando suavizado Savitzky-Golay a curvas de stress...
  -> Suavizado aplicado a 123 zonas
  -> Parametros: window=11, polyorder=3
FEATURE ENGINEERING COMPLETADO
Zonas procesadas: 123
Registros totales: 2,114

Columnas disponibles: ['zone_id', 'mean_delay_minute', 'avg_fail_rate', 'avg_staffing_affection', 'records_count', 'total_orders_matched'

## 3. Calculo de Umbrales por Zona

In [5]:
# Calcular umbrales usando curva suavizada
df_thresholds = calculate_zone_thresholds(df_stress)

# Mostrar primeras zonas
df_thresholds[['zone_name', 'city_name', 'low_delay', 'preventive', 
               'containment_I', 'containment_II', 'inoperable']].head(10)


CALCULO DE UMBRALES DE DELAY POR ZONA

Método: DELAY PROMEDIO PONDERADO POR BUCKET DE STRESS SUAVIZADO
Filtrando puntos con menos de 30 órdenes
Usando curva suavizada precalculada (Savitzky-Golay)
Truncando curva en el máximo de stress (delay_at_max)
Percentiles del stress suavizado (segun config):
  low_delay: stress en P0.0-P20.0 -> delay promedio ponderado
  preventive: stress en P20.0-P30.0 -> delay promedio ponderado
  containment_I: stress en P30.0-P45.0 -> delay promedio ponderado
  containment_II: stress en P45.0-P65.0 -> delay promedio ponderado
  inoperable: stress en P65.0-P100.0 -> delay promedio ponderado

  -> Umbrales calculados para 122 zonas

Estadisticas del truncado (delay del pico de stress):
  peak_delay: mean=15.6, min=9, max=24
  puntos ignorados (despues del pico): 101

Estadisticas de umbrales (minutos de mean_delay):
  low_delay: mean=4.7, std=1.4, range=[1, 8]
  preventive: mean=5.0, std=2.3, range=[0, 10]
  containment_I: mean=6.4, std=2.5, range=[0, 12]
  

,zone_name,city_name,low_delay,preventive,containment_I,containment_II,inoperable
0,Santa fe,Santa fe,3.679066,5.000000,0.345974,6.780533,9.745265
1,Nuevo cordoba,Cordoba,4.262048,6.000000,7.459049,9.272455,12.299108
2,Rosario,Rosario,4.430037,5.749613,6.805245,8.234536,11.422985
3,Cerro,Cordoba,4.215822,6.000000,6.452095,8.676443,12.462559
4,Martinez,Buenos aires,4.295646,5.808233,7.378634,9.872388,14.169058
5,Palermo,Buenos aires,3.975200,5.638097,7.894003,11.567921,15.665758
6,Mar del plata norte,Mar del plata,4.390335,5.961322,6.899948,9.158977,13.625418
7,Mendoza,Mendoza,5.714109,3.000000,6.977372,7.923432,9.590449
8,Microcentro,Buenos aires,5.354518,6.905761,8.457951,10.718735,15.149455
9,Caballito,Buenos aires,5.128591,7.477667,9.353337,11.724425,15.816827


## 4. Validacion con Sesiones Reales

In [6]:
# Query para obtener sesiones con su mean_delay
from google.cloud import bigquery

client = bigquery.Client(project="peya-argentina")
start_date = get_start_date()

query_sessions = f"""
WITH sessions_with_delay AS (
    SELECT
        s.session_id,
        s.zone_id,
        ROUND(md.congestion_delay) as mean_delay_minute
    FROM `{TABLES['sessions_zone']}` s
    INNER JOIN `{TABLES['mean_delay_report']}` md
        ON s.zone_id = md.zone_id
        AND TIMESTAMP_TRUNC(s.session_start_timestamp_local, MINUTE) = 
            TIMESTAMP_TRUNC(TIMESTAMP(DATETIME(md.report_timestamp, 'America/Argentina/Buenos_Aires')), MINUTE)
    WHERE s.report_date >= '{start_date}'
        AND md.report_timestamp >= '{start_date}'
        AND LOWER(md.country_code) = '{COUNTRY_CODE}'
        AND md.congestion_delay >= 0
)
SELECT
    zone_id,
    mean_delay_minute,
    COUNT(DISTINCT session_id) as session_count
FROM sessions_with_delay
GROUP BY zone_id, mean_delay_minute
ORDER BY zone_id, mean_delay_minute
"""

print(f"Extrayendo sesiones desde {start_date}...")
df_sessions = client.query(query_sessions).to_dataframe()
print(f"  -> {len(df_sessions):,} registros")
print(f"  -> {df_sessions['session_count'].sum():,} sesiones totales")

/Users/guidod/Desktop/Severity/venv/lib/python3.13/site-packages/google/auth/_default.py:114: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Extrayendo sesiones desde 2025-11-27...


/Users/guidod/Desktop/Severity/venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


  -> 4,303 registros
  -> 58,653,265 sesiones totales


In [7]:
# Clasificar sesiones por stage
def classify_session_stage(row, thresholds_dict):
    zone_id = row['zone_id']
    delay = row['mean_delay_minute']
    
    if zone_id not in thresholds_dict:
        return 'sin_umbral'
    
    t = thresholds_dict[zone_id]
    
    if delay >= t['inoperable']:
        return 'inoperable'
    elif delay >= t['containment_II']:
        return 'containment_II'
    elif delay >= t['containment_I']:
        return 'containment_I'
    elif delay >= t['preventive']:
        return 'preventive'
    else:
        return 'low_delay'

# Preparar umbrales para export
df_export = export_thresholds_for_config(df_thresholds)
thresholds_dict = df_export.set_index('zone_id')[
    ['low_delay', 'preventive', 'containment_I', 'containment_II', 'inoperable']
].to_dict('index')

# Clasificar sesiones
df_sessions['stage'] = df_sessions.apply(lambda r: classify_session_stage(r, thresholds_dict), axis=1)

# Calcular porcentajes por zona
total_by_zone = df_sessions.groupby('zone_id')['session_count'].sum().reset_index()
total_by_zone.columns = ['zone_id', 'total_sessions']

sessions_by_stage = df_sessions.groupby(['zone_id', 'stage'])['session_count'].sum().reset_index()
sessions_by_stage = sessions_by_stage.merge(total_by_zone, on='zone_id')
sessions_by_stage['pct'] = (sessions_by_stage['session_count'] / sessions_by_stage['total_sessions'] * 100).round(2)

# Pivotar
df_validation = sessions_by_stage.pivot(index='zone_id', columns='stage', values='pct').fillna(0)
stage_order = ['low_delay', 'preventive', 'containment_I', 'containment_II', 'inoperable', 'sin_umbral']
df_validation = df_validation[[c for c in stage_order if c in df_validation.columns]].reset_index()

# Agregar info de zona
df_validation = df_validation.merge(df_thresholds[['zone_id', 'zone_name', 'city_name']], on='zone_id', how='left')
df_validation = df_validation.merge(total_by_zone, on='zone_id')

print("\nDistribucion de sesiones por stage (primeras 15 zonas):")
cols_show = ['zone_name', 'city_name', 'total_sessions', 'low_delay', 'preventive', 
             'containment_I', 'containment_II', 'inoperable']
print(df_validation[[c for c in cols_show if c in df_validation.columns]].head(15).to_string(index=False))


Distribucion de sesiones por stage (primeras 15 zonas):
          zone_name     city_name  total_sessions  low_delay  preventive  containment_I  containment_II  inoperable
           Santa fe      Santa fe          949777      21.64         0.0          38.96           29.54        9.85
      Nuevo cordoba       Cordoba          677310      14.28        9.83          28.25           29.74       17.89
            Rosario       Rosario         2399503      29.84       18.97          13.87           22.88       14.44
              Cerro       Cordoba          435804      43.65         0.0          40.16           10.04        6.15
           Martinez  Buenos aires          866221      10.61        7.03          28.56           26.23       27.57
            Palermo  Buenos aires         2240319      13.33       20.92          34.53           20.53        10.7
Mar del plata norte Mar del plata         1484977       5.27        7.47          33.92           39.49       13.85
            Men

## 5. Generar PDF de Diagnostico

In [8]:
# Generar PDF con curva suavizada para cada zona
pdf_path = '../output/diagnostico_zonas.pdf'

print(f"Generando PDF de diagnostico...")
print(f"Total de zonas a procesar: {len(df_thresholds)}")

# Colores por stage
stage_colors = {
    'low_delay': 'green', 
    'preventive': 'gold', 
    'containment_I': 'orange', 
    'containment_II': 'orangered', 
    'inoperable': 'darkred'
}
stages = ['low_delay', 'preventive', 'containment_I', 'containment_II', 'inoperable']

with PdfPages(pdf_path) as pdf:
    zones_sorted = df_thresholds.sort_values('zone_name')

    for i, (_, zone_row) in enumerate(zones_sorted.iterrows()):
        zone_id = zone_row['zone_id']
        zone_name = zone_row['zone_name']
        city_name = zone_row['city_name']

        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        fig.suptitle(f'{zone_name} ({city_name}) - Zone ID: {zone_id}', fontsize=14, fontweight='bold')

        # === SUBPLOT 1: Curva de Stress Suavizada ===
        ax1 = axes[0]
        zone_data = df_stress[df_stress['zone_id'] == zone_id].sort_values('mean_delay_minute')

        if len(zone_data) > 0:
            delays = zone_data['mean_delay_minute'].values
            stress_orig = zone_data['stress_score_normalized'].values
            
            # Puntos originales en gris
            ax1.scatter(delays, stress_orig, alpha=0.35, c='gray', s=30, 
                       edgecolors='black', linewidths=0.3, label='Datos originales')
            
            # Curva suavizada en azul
            if 'stress_smoothed' in zone_data.columns:
                stress_smooth = zone_data['stress_smoothed'].values
                ax1.plot(delays, stress_smooth, 'b-', linewidth=2.5, 
                        label='Curva suavizada', zorder=10)
                
                # Marcar minimo (triangulo verde)
                if 'delay_at_min' in zone_data.columns:
                    delay_min = zone_data['delay_at_min'].iloc[0]
                    stress_min = zone_data['stress_smooth_min'].iloc[0]
                    ax1.scatter([delay_min], [stress_min], color='green', s=200, 
                               marker='v', zorder=15, edgecolors='black', linewidths=1.5,
                               label=f'Min: {delay_min:.0f} min')
                
                # Marcar maximo (triangulo rojo)
                if 'delay_at_max' in zone_data.columns:
                    delay_max = zone_data['delay_at_max'].iloc[0]
                    stress_max_val = zone_data['stress_smooth_max'].iloc[0]
                    ax1.scatter([delay_max], [stress_max_val], color='red', s=200, 
                               marker='^', zorder=15, edgecolors='black', linewidths=1.5,
                               label=f'Max: {delay_max:.0f} min')

            # Lineas verticales de umbrales
            for stage, color in stage_colors.items():
                if stage in zone_row.index and pd.notna(zone_row[stage]):
                    ax1.axvline(x=zone_row[stage], color=color, linestyle='--',
                               alpha=0.8, linewidth=2, label=f'{stage}: {zone_row[stage]:.1f}')

            ax1.set_xlabel('Mean Delay (minutos)', fontsize=10)
            ax1.set_ylabel('Stress Score Normalizado (0-100)', fontsize=10)
            ax1.set_title('Curva de Stress Suavizada', fontsize=11)
            ax1.set_ylim(-5, 110)
            ax1.legend(fontsize=7, loc='upper left')
            ax1.grid(True, alpha=0.3)

        # === SUBPLOT 2: Distribucion de Sesiones ===
        ax2 = axes[1]
        zone_val = df_validation[df_validation['zone_id'] == zone_id]

        if len(zone_val) > 0:
            colors = ['green', 'gold', 'orange', 'orangered', 'darkred']
            values = [zone_val[s].values[0] if s in zone_val.columns else 0 for s in stages]

            left = 0
            for stage, val, color in zip(stages, values, colors):
                ax2.barh(0, val, left=left, color=color, height=0.5, edgecolor='black', linewidth=0.5)
                if val > 5:
                    ax2.text(left + val/2, 0, f'{val:.1f}%', ha='center', va='center', 
                            fontsize=8, fontweight='bold')
                left += val

            ax2.set_xlim(0, 100)
            ax2.set_xlabel('% de Sesiones', fontsize=10)
            ax2.set_yticks([])
            ax2.set_title('Distribucion de Sesiones por Stage', fontsize=11)
            
            legend_elements = [Patch(facecolor=c, edgecolor='black', label=f'{s}: {v:.1f}%') 
                              for s, c, v in zip(stages, colors, values)]
            ax2.legend(handles=legend_elements, fontsize=7, loc='upper right', ncol=1)
            ax2.grid(True, alpha=0.3, axis='x')
        else:
            ax2.text(0.5, 0.5, 'Sin datos de sesiones', ha='center', va='center', transform=ax2.transAxes)

        plt.tight_layout()
        pdf.savefig(fig, bbox_inches='tight')
        plt.close(fig)
        
        if (i + 1) % 20 == 0:
            print(f"  Procesadas {i + 1}/{len(zones_sorted)} zonas...")

print(f"\n{'='*60}")
print(f"PDF generado: {pdf_path}")
print(f"Total de zonas: {len(zones_sorted)}")
print(f"{'='*60}")

Generando PDF de diagnostico...
Total de zonas a procesar: 122
  Procesadas 20/122 zonas...
  Procesadas 40/122 zonas...
  Procesadas 60/122 zonas...
  Procesadas 80/122 zonas...
  Procesadas 100/122 zonas...
  Procesadas 120/122 zonas...

PDF generado: ../output/diagnostico_zonas.pdf
Total de zonas: 122


## 6. Exportar Umbrales

In [9]:
# Mostrar umbrales finales
print(f"Zonas listas para exportar: {len(df_export)}")
df_export.head(10)

Zonas listas para exportar: 122


,zone_id,zone_name,city_id,city_name,low_delay,preventive,containment_I,containment_II,inoperable
0,3,Santa fe,2,Santa fe,4,5,5,7,10
1,4,Nuevo cordoba,3,Cordoba,4,6,7,9,12
2,5,Rosario,4,Rosario,4,6,7,8,11
3,9,Cerro,3,Cordoba,4,6,6,9,12
4,10,Martinez,1,Buenos aires,4,6,7,10,14
5,13,Palermo,1,Buenos aires,4,6,8,12,16
6,15,Mar del plata norte,200,Mar del plata,4,6,7,9,14
7,16,Mendoza,201,Mendoza,6,6,7,8,10
8,25,Microcentro,1,Buenos aires,5,7,8,11,15
9,26,Caballito,1,Buenos aires,5,7,9,12,16


In [10]:
# Guardar a CSV
df_export.to_csv('../output/zone_thresholds.csv', index=False)
print("Exportado a ../output/zone_thresholds.csv")

Exportado a ../output/zone_thresholds.csv
